In [1]:
import sys
sys.path.append("/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src")
from pathlib import Path
import pandas as pd

from transformers import AutoTokenizer

from data_utils import clean, encode_multilabel, get_folds, encode_df

In [2]:
data_dir = Path("/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/dataset")

data_path = data_dir / "ruddit_data" / "ruddit_with_text.csv"

In [3]:
data = pd.read_csv(data_path)
data = data[['comment_id', 'txt', 'offensiveness_score']]
data = data.rename(columns={'txt': 'text', 'offensiveness_score': 'label'})
print(data.shape)
data.head()

(5838, 3)


,comment_id,text,label
0,cza1q49,> The difference in average earnings between m...,-0.083
1,cza1wdh,"The myth is that the ""gap"" is entirely based o...",-0.022
2,cza23qx,[deleted],0.167
3,cza2bw8,The assertion is that women get paid less for ...,-0.146
4,cza2iji,You said in the OP that's not what they're mea...,-0.083


In [4]:
data = clean(data.copy(), col_name='text', min_len=15)

# special clean for ruddit
data.loc[:, 'text'] = data['text'].str.strip('>')
data.loc[:, 'text'] = data['text'].str.strip()

print(data.shape)
data.head()

(5707, 3)


/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src/data_utils.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_name] = df[col_name].str.replace('https?://\S+|www\.\S+', ' social medium ')
/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src/data_utils.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col_name] = df[col_name].str.replace('\s+', ' ')


,comment_id,text,label
0,cza1q49,The difference in average earnings between men...,-0.083
1,cza1wdh,"The myth is that the ""gap"" is entirely based o...",-0.022
2,cza2bw8,The assertion is that women get paid less for ...,-0.146
3,cza2iji,You said in the OP that's not what they're mea...,-0.083
4,cza2jj3,Men and women are not payed less for the same ...,-0.042


#### Encoding

In [5]:
tokenizer_type = 'roberta-base'  # change if needed

tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)

In [6]:
data = encode_df(data.copy(), tokenizer,
                 col_name='text', max_length=256, drop=True)

100%|██████████| 5707/5707 [00:01<00:00, 3911.52it/s]
/Users/avgalichin/Documents/kaggle/Jigsaw2022/Jigsaw18/src/data_utils.py:78: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(col_name, 1, inplace=True)


In [7]:
data.to_csv(data_dir / 'ruddit_data' / 'roberta_ruddit_data.csv', index=False)